## Time Series Forecasting with Multiple Models

### 0. Import libraries and setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_loading import load_m4_financial
from models import (
    ARIMAModel,
    HoltWintersModel,
    RNNModel,
    LSTMModel,
    XGBoostModel,
    InformerModel,
    TFTModel
)

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load M4 Financial data

In [ ]:
print("Loading M4 financial data...")
load_m4_financial()  # This will download the data if it's not already present

In [ ]:
# For this notebook, we'll focus on the Monthly financial data
from datasetsforecast.m4 import M4
m4_info = M4.info()
print(f"M4 dataset info: {m4_info}")

In [ ]:
# Load monthly financial data
monthly_data = M4.load(directory='./data', group='Monthly', cache=True)
print(f"Dataset shape: {monthly_data['dataset'].shape}")
print(f"Number of financial time series: {len(monthly_data['dataset'])}")

## 2. Exploratory Data Analysis

In [ ]:
# Extract a few financial time series for analysis
financial_series = monthly_data['dataset'].iloc[:5]  # First 5 series

plt.figure(figsize=(15, 10))
for i, series in enumerate(financial_series):
    plt.subplot(len(financial_series), 1, i+1)
    plt.plot(series)
    plt.title(f"Financial Time Series {i+1}")
    plt.tight_layout()
plt.show()

In [ ]:
print("Basic statistics of the first 5 financial time series:")
for i, series in enumerate(financial_series):
    print(f"Series {i+1}:")
    print(f"  Length: {len(series)}")
    print(f"  Mean: {np.mean(series):.2f}")
    print(f"  Std: {np.std(series):.2f}")
    print(f"  Min: {np.min(series):.2f}")
    print(f"  Max: {np.max(series):.2f}")
    print()

In [ ]:
# Check for seasonality and trend in one example series
from statsmodels.tsa.seasonal import seasonal_decompose

# Select a single series with sufficient data points
example_series = financial_series.iloc[0]
if len(example_series) >= 24:  # Need at least 2x the seasonal period
    decomposition = seasonal_decompose(example_series, model='multiplicative', period=12)
    
    plt.figure(figsize=(12, 10))
    plt.subplot(4, 1, 1)
    plt.plot(example_series)
    plt.title('Original')
    
    plt.subplot(4, 1, 2)
    plt.plot(decomposition.trend)
    plt.title('Trend')
    
    plt.subplot(4, 1, 3)
    plt.plot(decomposition.seasonal)
    plt.title('Seasonality')
    
    plt.subplot(4, 1, 4)
    plt.plot(decomposition.resid)
    plt.title('Residuals')
    
    plt.tight_layout()
    plt.show()

## 3. Data Preparation

In [ ]:
# Since we have multiple time series, let's select one for detailed analysis
ts_idx = 0  # We'll use the first time series
selected_series = financial_series.iloc[ts_idx].dropna()

In [ ]:
# Define the forecast horizon
forecast_horizon = 12  # For 12 months ahead forecasting

# Train-test split
train_size = len(selected_series) - forecast_horizon
train_data = selected_series[:train_size]
test_data = selected_series[train_size:]

print(f"Training data length: {len(train_data)}")
print(f"Test data length: {len(test_data)}")

In [ ]:
# Scale data for neural network models
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
test_scaled = scaler.transform(test_data.values.reshape(-1, 1))

In [ ]:
# Prepare windowed dataset for neural networks
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [ ]:
# Create window size for sequential models
window_size = 12  # Use 12 months of data to predict the next month
X_train, y_train = create_sequences(train_scaled, window_size)
# For testing we'll use the full test data set separately

# Reshape for RNN/LSTM models
X_train_rnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

print(f"X_train shape for RNN/LSTM: {X_train_rnn.shape}")
print(f"y_train shape: {y_train.shape}")

## 4. Model Training & Evaluation

In [ ]:
# Define a function to evaluate models
def evaluate_forecast(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, mape

# Dictionary to store results
model_results = {}

#### 4.1 Train and evaluate ARIMA model

In [ ]:
print("\nTraining ARIMA model...")
arima_model = ARIMAModel(seasonal=True, m=12)  # Monthly data with yearly seasonality
arima_model.fit(train_data)
arima_forecast = arima_model.forecast(forecast_horizon)
arima_model.summary()

In [ ]:
# Evaluate
mae, rmse, mape = evaluate_forecast(test_data, arima_forecast)
model_results['ARIMA'] = {
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'forecast': arima_forecast
}
print(f"ARIMA - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

#### 4.2 Train and evaluate Holt-Winters model

In [ ]:
print("\nTraining Holt-Winters model...")
hw_model = HoltWintersModel(seasonal='mul', seasonal_periods=12, trend='add')
hw_model.fit(train_data)
hw_forecast = hw_model.forecast(forecast_horizon)
hw_model.summary()

In [ ]:
# Evaluate
mae, rmse, mape = evaluate_forecast(test_data, hw_forecast)
model_results['Holt-Winters'] = {
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'forecast': hw_forecast
}
print(f"Holt-Winters - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

#### 4.3 Train and evaluate XGBoost model

In [1]:
# We need to prepare the data differently for XGBoost
def create_features(series, window=12):
    """Create features for XGBoost based on lag values"""
    df = pd.DataFrame(series)
    df.columns = ['y']
    
    # Add lag features
    for i in range(1, window+1):
        df[f'lag_{i}'] = df['y'].shift(i)
    
    # Drop rows with NaN values
    df = df.dropna()
    
    return df

# Prepare data for XGBoost
xgb_data = create_features(train_data, window=12)  # Use 12 months of lags
X_xgb = xgb_data.drop('y', axis=1)
y_xgb = xgb_data['y']


Training XGBoost model...


NameError: name 'train_data' is not defined

In [ ]:
print("\nTraining XGBoost model...")
# Initialize and train XGBoost model
xgb_model = XGBoostModel(n_estimators=100, max_depth=3, learning_rate=0.1)
xgb_model.fit(X_xgb, y_xgb)
xgb_model.summary()

In [ ]:
# Generate forecasts one step at a time
xgb_forecast = []
last_window = list(train_data[-12:])  # Last 12 months of training data

for i in range(forecast_horizon):
    # Create features for next prediction
    features = np.array(last_window)
    features = features.reshape(1, -1)  # Reshape for prediction
    
    # Make prediction
    prediction = xgb_model.predict(features)
    xgb_forecast.append(prediction[0])
    
    # Update window
    last_window.pop(0)
    last_window.append(prediction[0])

In [ ]:
# Evaluate
mae, rmse, mape = evaluate_forecast(test_data, xgb_forecast)
model_results['XGBoost'] = {
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'forecast': xgb_forecast
}
print(f"XGBoost - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

#### 4.4 Train and evaluate RNN model

In [ ]:
print("\nTraining RNN model...")
# Define input shape for RNN
input_shape = (window_size, 1)  # (timesteps, features)

In [ ]:
# Initialize model
rnn_model = RNNModel(input_shape=input_shape, hidden_dim=32, output_dim=1)
# Train model
rnn_history = rnn_model.fit(
    X_train_rnn,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)
rnn_model.summary()


In [ ]:
# Generate forecasts one step at a time
rnn_forecast_scaled = []
last_window = train_scaled[-window_size:].reshape(1, window_size, 1)

for i in range(forecast_horizon):
    # Make prediction
    prediction = rnn_model.predict(last_window)
    rnn_forecast_scaled.append(prediction[0][0])
    
    # Update window
    last_window = np.append(last_window[:, 1:, :], 
                           prediction.reshape(1, 1, 1), 
                           axis=1)

# Inverse transform to get original scale
rnn_forecast = scaler.inverse_transform(np.array(rnn_forecast_scaled).reshape(-1, 1)).flatten()

In [ ]:
# Evaluate
mae, rmse, mape = evaluate_forecast(test_data, rnn_forecast)
model_results['RNN'] = {
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'forecast': rnn_forecast
}
print(f"RNN - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

#### 4.5 Train and evaluate LSTM model

In [ ]:
print("\nTraining LSTM model...")
# Initialize model
lstm_model = LSTMModel(input_shape=input_shape, hidden_dim=32, output_dim=1)

In [ ]:
# Train model
lstm_history = lstm_model.fit(
    X_train_rnn,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)
lstm_model.summary()

In [ ]:
# Generate forecasts one step at a time
lstm_forecast_scaled = []
last_window = train_scaled[-window_size:].reshape(1, window_size, 1)

for i in range(forecast_horizon):
    # Make prediction
    prediction = lstm_model.predict(last_window)
    lstm_forecast_scaled.append(prediction[0][0])
    
    # Update window
    last_window = np.append(last_window[:, 1:, :], 
                           prediction.reshape(1, 1, 1), 
                           axis=1)

# Inverse transform to get original scale
lstm_forecast = scaler.inverse_transform(np.array(lstm_forecast_scaled).reshape(-1, 1)).flatten()

In [ ]:
# Evaluate
mae, rmse, mape = evaluate_forecast(test_data, lstm_forecast)
model_results['LSTM'] = {
    'mae': mae,
    'rmse': rmse,
    'mape': mape,
    'forecast': lstm_forecast
}
print(f"LSTM - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

#### 4.6 Training and evaluating Informer model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
torch_available = True
    
# Create a PyTorch dataset for time series
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size):
        self.data = torch.FloatTensor(data)
        self.window_size = window_size
            
    def __len__(self):
        return len(self.data) - self.window_size
            
    def __getitem__(self, index):
        x = self.data[index:index+self.window_size]
        y = self.data[index+self.window_size]
        return x, y

In [ ]:
# Define parameters for Informer
input_dim = 1  # univariate time series
d_model = 64   # embedding dimension
d_ff = 128     # feed-forward network dimension
n_heads = 4    # number of attention heads
e_layers = 2   # number of encoder layers
d_layers = 1   # number of decoder layers
dropout = 0.1

In [ ]:
# Convert to PyTorch tensors
train_tensor = torch.FloatTensor(train_scaled)

# Encoder input: Use the last window_size points from training data
enc_input = train_tensor[-window_size:].unsqueeze(0).unsqueeze(-1)  # Shape: [1, window_size, 1]

# Decoder input: Use a zero tensor as the seed (we'll generate step by step)
dec_input = torch.zeros((1, forecast_horizon, 1))  # Shape: [1, forecast_horizon, 1]

In [ ]:
# Initialize the Informer model
informer_model = InformerModel(
    input_dim=input_dim,
    d_model=d_model,
    d_ff=d_ff,
    n_heads=n_heads,
    e_layers=e_layers,
    d_layers=d_layers,
    dropout=dropout,
    out_dim=1
)
    
# Training loop for Informer (normally this would use a proper dataset, but we'll simplify)
# Create a dataset from training data
train_dataset = TimeSeriesDataset(train_scaled, window_size)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(informer_model.parameters(), lr=0.001)

# Training loop
epochs = 30
informer_model.train()

for epoch in range(epochs):
        epoch_loss = 0
        for batch_x, batch_y in train_loader:
            # Add feature dimension if not present
            if batch_x.dim() == 2:
                batch_x = batch_x.unsqueeze(-1)
            
            # Create a simple decoder input (we use zeros as placeholder)
            batch_size = batch_x.size(0)
            decoder_input = torch.zeros((batch_size, 1, 1))
            
            # Forward pass
            optimizer.zero_grad()
            
            # In a real implementation, we'd use different approaches for 
            # encoder and decoder inputs. For simplicity, we'll use a hacky approach here.
            
            # Simulate the Informer forward pass with our model
            # Normally: output = informer_model(batch_x, decoder_input)
            # But our implementation expects specific encoder/decoder inputs
            
            # Here we simplify significantly - in practice you'd need to set up 
            # proper encoder/decoder sequences
            enc_in = batch_x
            dec_in = decoder_input
            
            # For our simplified model training
            # We'll just predict a single step for now
            output = informer_model(enc_in, dec_in)
            
            # Reshape batch_y to match output dimensions
            target = batch_y.unsqueeze(-1).unsqueeze(-1)
            
            # Calculate loss
            loss = criterion(output[:, -1, :], target)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        if epoch % 5 == 0:
            print(f"Epoch {epoch} | Loss: {epoch_loss/len(train_loader):.6f}")

#### 4.7 Training and evaluating Autoformer model

#### 4.8 Training and evaluating TFT model

## 5. Model comparison

In [ ]:
# Compare metrics across all models
metrics_df = pd.DataFrame({
    'MAE': [results['mae'] for model, results in model_results.items()],
    'RMSE': [results['rmse'] for model, results in model_results.items()],
    'MAPE (%)': [results['mape'] for model, results in model_results.items()]
}, index=model_results.keys())

print("\nModel Comparison:")
print(metrics_df)

In [ ]:
# Bar plot for model comparison
plt.figure(figsize=(14, 10))

# MAE subplot
plt.subplot(3, 1, 1)
metrics_df['MAE'].plot(kind='bar', color='skyblue')
plt.title('Mean Absolute Error (MAE)')
plt.grid(axis='y', alpha=0.3)

# RMSE subplot
plt.subplot(3, 1, 2)
metrics_df['RMSE'].plot(kind='bar', color='salmon')
plt.title('Root Mean Squared Error (RMSE)')
plt.grid(axis='y', alpha=0.3)

# MAPE subplot
plt.subplot(3, 1, 3)
metrics_df['MAPE (%)'].plot(kind='bar', color='lightgreen')
plt.title('Mean Absolute Percentage Error (MAPE)')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Visualization of Forecasts

In [ ]:
plt.figure(figsize=(12, 8))

# Plot actual data
plt.plot(range(len(train_data)), train_data, 'b-', label='Historical Data')
plt.plot(range(len(train_data), len(train_data) + len(test_data)), test_data, 'k-', label='Actual Future')

# Plot all model forecasts
offset = len(train_data)
for model_name, results in model_results.items():
    plt.plot(range(offset, offset + forecast_horizon), results['forecast'], '--', label=f'{model_name} Forecast')

plt.title('Time Series Forecasting Comparison')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## 7. Conclusion

In [ ]:
# Find the best model based on MAPE
best_model = metrics_df['MAPE (%)'].idxmin()
print(f"\nThe best performing model is: {best_model}")
print(f"It achieved a MAPE of {metrics_df.loc[best_model, 'MAPE (%)']: .2f}%")